# INSERT INTO DB

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')

In [2]:
reviews = pd.read_json('./prc_data/reviews_prc_4.json')

In [10]:
import sqlite3
conn = sqlite3.connect('./apparel.db')
cur = conn.cursor()

### item table

In [ ]:
items = pd.read_json('./items_prc.json')
items.info()

In [8]:
# 데이터 입력에 필요한 형태로 변환
items = items[['item_id', 'cat', 'name', 'price', 'img', 'link']]
items.item_id = items.item_id.astype(str)

In [9]:
for row in items.itertuples(index=False, name=None):
    cur.execute('''
                INSERT INTO item VALUES (?, ?, ?, ?, ?, ?); ''', row)
    conn.commit()

### reviews => user, review로 나누기

In [3]:
reviews.head()

,품번,아이디,별점,키,몸무게,평소사이즈,선택1,리뷰,사이즈,color,색상
0,630,핫핑스탭,-1,161~165,61 ~ 65,2.0,None,[스텝후기] 넉넉한 오버핏에 여유있는 기장감으로 편안하게 착용되고 상체군살이 ...,None,None,None
1,630,핫핑스탭,-1,161~165,51 ~ 55,NaN,None,[스탭후기] 완전 박시하게 떨어지는 핏으로 체형커버가 잘 되요 부해보이지 않고 편...,F,None,먹색
2,630,스탭리뷰,-1,None,None,NaN,None,[스탭후기] 많이 여유있는 프리 사이즈로 많은 분들께서 부담없이 착용할 수 있어...,F,None,차콜
3,630,핫핑스텝,-1,161~165,65~,NaN,None,[스탭후기] 루즈한 어깨핏에 힙을 충분히 덮는 길이감으로 상체 라인 커버하기 좋은...,F,None,차콜
4,630,128470272,5,156~160,61 ~ 65,3.0,None,예~전에 차콜색 사고 맘에 들어서 이번엔 흰색 샀는데 사이즈가 원래도 컸는데 완전 ...,F,None,화이트


In [6]:
reviews.replace({np.NaN: None}, inplace=True)

In [7]:
# user, review 정보 나누기
user = reviews[['아이디', '키', '몸무게', '평소사이즈']]
review = reviews[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]

#### user Table

In [8]:
for col in ['키', '몸무게', '평소사이즈']:
    print(col)
    print(user[col].unique())

키
['161~165' None '156~160' '166~170' '171~175' '151~155' '176이상' '146~150'
 '139이하' '140~145']
몸무게
['61 ~ 65' '51 ~ 55' None '65~' '56 ~ 60' '46 ~ 50' '~45']
평소사이즈
[2.0 None 3.0 5.0 4.0 1.0]


In [9]:
user_dup = user.loc[user['아이디'].duplicated()].index
user.drop(user_dup, axis=0, inplace=True)
user.reset_index(drop=True, inplace=True)
user.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12425 entries, 0 to 12424
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     12425 non-null  object
 1   키       9497 non-null   object
 2   몸무게     8627 non-null   object
 3   평소사이즈   8574 non-null   object
dtypes: object(4)
memory usage: 388.4+ KB


In [11]:
# 아이디 중복 확인 후 데이터 입력
for row in user.itertuples(name=None, index=False):
    # cur.execute(f'''
    #             INSERT INTO user VALUES(?, ?, ?, ?)
    #             SELECT * FROM user
    #             WHERE NOT EXISTS
    #             (SELECT 1 FROM USER
    #             WHERE id == {row[0]});''', row)
    cur.execute('''
                INSERT OR IGNORE
                INTO user VALUES(?,?,?,?);''', row) # 아이디 중복 제외
    
    conn.commit()

In [12]:
cur.execute('''
            SELECT COUNT(*) FROM user;''')
print(cur.fetchone())

(50938,)


#### review Table

In [13]:
review = review[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]
review_dup = review.loc[review['리뷰'].duplicated()==True].index
review.drop(review_dup, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
len(review)

18149

In [14]:
review.duplicated().sum()

0

In [15]:
# '''
# review table 줄바꿈 제거 필요
# '''
# cur.execute('''
#             UPDATE review
#             SET comment = replace(comment, '\n', ' ');''')
# conn.commit()

In [15]:
for row in review.itertuples(name=None, index=True):
    cur.execute('''
                INSERT OR IGNORE
                INTO REVIEW VALUES(?,?,?,?,?,?,?);''', row)
    conn.commit()

### size Table

In [17]:
# cur.execute('''
#             CREATE TABLE size(
#                 id VARCHAR(1) NOT NULL,
#                 size VARCHAR(10) NOT NULL,
#                 PRIMARY KEY(id));''')
size_id = ['1', '2', '3', '4', '5', 'F']
size = ['XS-S','M', 'L', 'XL', '2XL-3XL', 'F']
s_dict = dict(zip(size_id, size))
for row in s_dict.items():
    cur.execute(f'''
                INSERT INTO size VALUES (?,?);''', row)
    conn.commit()

## 연결 해제

In [16]:
cur.close()
conn.close()